In [1]:
#!/usr/bin/env python3

import paho.mqtt.client as mqtt
import paho.mqtt.client as mqtt
import time
from timeit import default_timer as timer
from datetime import timedelta
import multiprocessing

SERVER_IP = "192.168.29.149"
PORT = 1883
KEEP_ALIVE = 60
QOS = 0
QOS1 = 1
QOS2 = 2
RETAIN = False

SIGNATURE = "end"

max_publishers = 2
max_client_per_publisher = 15

def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    client.subscribe("topic/#")

def on_message(client, userdata, msg):
    if SIGNATURE in msg.payload.decode():
        #print(end - float(msg.payload.decode().split(":")[1]))
        client.disconnect()

# stops latency calculator loop.
def on_disconnect(client, userdata,rc=0):
    client.loop_stop()
    
# this is the publisher thread
def client_listner(publisher_id):
    print("Publisher [" + str(publisher_id) + "] started...\r")
    
    clients=[]
    #mqtt.Client.connected_flag=False
    #create clients
    for i in range(max_client_per_publisher):
        cname="Client"+str(publisher_id)+str(i)
        client= mqtt.Client(cname)
        #client= mqtt.Client()
        clients.append(client)

    for client in clients:
        client.on_connect = on_connect
        client.on_message = on_message
        client.on_disconnect = on_disconnect
        client.connect(SERVER_IP, PORT, KEEP_ALIVE)
    
    for client in clients:
        client.loop_forever()
    
    
if __name__ == "__main__":
    # publishers
    proc_list = []  
    for n in range(max_publishers):
        # creating publisher process
        print("Registering Client[" + str(n) + "]..\r")
        p = multiprocessing.Process(target=client_listner, args=(n,))
        proc_list.append(p)

    # schedule all procs
    for proc in proc_list:
        proc.start()

    # joins procs
    for proc in proc_list:
        proc.join()

    print("Done")

Registering Client[0]..
Registering Client[1]..
Publisher [0] started...
Publisher [1] started...
Connected with result code 0
Connected with result code 0
Connected with result code 0
Connected with result code 0


Process Process-2:
Process Process-1:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/biswa/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/biswa/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/biswa/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/biswa/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-1-3cc296fe0a3c>", line 56, in client_listner
    client.loop_forever()
  File "<ipython-input-1-3cc296fe0a3c>", line 56, in client_listner
    client.loop_forever()
  File "/home/biswa/anaconda3/lib/python3.7/site-packages/paho/mqtt/client.py", line 1481, in loop_forever
    rc = self.loop(timeout, max_packets)
  File "/home/biswa/anaconda3/lib/python3.7/site-packages/paho

KeyboardInterrupt: 